# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [8]:
#link to the dataset: https://data.ny.gov/Government-Finance/Lottery-Aid-to-Education-Beginning-2002/9ypc-vjiq/data

#Load the data with pandas
import pandas as pd
df = pd.read_csv('lottery_aid_education.csv')
df.head()


,Beginning Fiscal Year,Ending Fiscal Year,County,School District,Amount of Aid
0,2002,2003,Albany,Albany,7242297.04
1,2002,2003,Albany,Berne-Knox-Westerlo,874046.94
2,2002,2003,Albany,Bethlehem,2786954.71
3,2002,2003,Albany,Cohoes,1820297.94
4,2002,2003,Albany,Green Island,241859.93


## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [12]:
%%R
require('tidycensus')
require('tidyverse')

Loading required package: tidycensus


In [19]:
%%R
data_acs <- get_acs(geography = "county", 
                      state='NY',
                      variables = c(
                        pop = "B01003_001",
                        male_grade_1to4 = "B14002_011",
                        male_grade_5to8 = "B14002_014",
                        male_grade_9to12 = "B14002_017",
                        female_grade_1to4 = "B14002_035",
                        female_grade_5to8 = "B14002_038",
                        female_grade_9to12 = "B14002_041",
                        pov_grade_1to4 = "B14006_006",
                        pov_grade_5to8 = "B14006_007",
                        pov_grade_9to12 = "B14006_008"
                        ), 
                      year = 2021,
                      survey="acs5")

Getting data from the 2017-2021 5-year ACS
Using FIPS code '36' for state 'NY'


In [15]:
%%R
population
#write population to csv
write_csv(population, "population.csv")


In [16]:
%%R

lottery_aid <- read_csv('lottery_aid_education.csv')
population <- read_csv('population.csv')

Rows: 14294 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): County, School District
dbl (3): Beginning Fiscal Year, Ending Fiscal Year, Amount of Aid

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 62 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): NAME, variable
dbl (2): GEOID, estimate
lgl (1): moe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R
#merge the two datasets, population and lottery_aid, by county
lottery_aid <- lottery_aid %>%
  left_join(population, by = c("county" = "NAME")) %>%
  mutate(education_aid_per_capita = aid / pop)

## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).